In [1]:
import os
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import gcsfs
import pyarrow.feather as feather
from motif_utils import *
import netsci.metrics.motifs as nsm
import json

## Loading the graph

In [2]:
DATASET = 'fafb_783'
REGION = 'central_complex'
NETWORK = f'{DATASET}_{REGION}'
print(NETWORK)

EDGE_LOCAL_PATH = f'data/{NETWORK}/simple_edgelist.feather'
SYN_LOCAL_PATH = f'data/{NETWORK}/synapses.feather'

edge_list_df = pd.read_feather(EDGE_LOCAL_PATH)
synapses_df = pd.read_feather(SYN_LOCAL_PATH)

fafb_783_central_complex


In [ ]:
print(f'✓ Loaded {len(synapses_df):,} synapses')
G_connectome = nx.DiGraph()

# Add edges (multiple synapses between same neurons become edge weight)
edge_counts = synapses_df.groupby(['pre', 'post']).size().reset_index(name='weight')
for _, row in edge_counts.iterrows():
    G_connectome.add_edge(row['pre'], row['post'], weight=row['weight'])

print(f'✓ Built connectome graph')
print(f'  Nodes: {G_connectome.number_of_nodes():,}')
print(f'  Edges: {G_connectome.number_of_edges():,}')
print(f'  Density: {nx.density(G_connectome):.4f}')

mat_sparse = nx.to_scipy_sparse_array(G_connectome)
bin_mat_sparse = mat_sparse.copy()
bin_mat_sparse[bin_mat_sparse > 1] = 1

# Create mapping between node IDs and matrix indices
node_list = list(G_connectome.nodes())
node_id_to_index = {node_id: idx for idx, node_id in enumerate(node_list)}
index_to_node_id = {idx: node_id for idx, node_id in enumerate(node_list)}

## Basic validation

In [ ]:
out_degree = G_connectome.out_degree(index_to_node_id[1290])
out_degree_mat = bin_mat_sparse[1290, :].sum()
print(out_degree, out_degree_mat)

for node in tqdm(G_connectome.nodes):
    assert G_connectome.out_degree(node) == bin_mat_sparse[node_id_to_index[node], :].sum()
    assert G_connectome.in_degree(node) == bin_mat_sparse[:, node_id_to_index[node]].sum()
    break

In [ ]:
def stats(conn_mat: np.ndarray):
    print(f'total connections (synapses): {np.sum(conn_mat)}')
    print('total number of elements in the connectivity matrix (N^2):', conn_mat.size)
    print(f"Non-zero elements: {np.count_nonzero(conn_mat)}")
    print(f"Percentage of non-zero elements: {np.count_nonzero(conn_mat) / conn_mat.size * 100:.2f}%")

    max_post_idx, max_pre_idx = np.unravel_index(conn_mat.argmax(), conn_mat.shape)
    print(f'Max synapses between a single pair of neurons: {conn_mat[max_post_idx, max_pre_idx]} (from {max_pre_idx} to {max_post_idx})')


stats(mat_sparse.toarray())
print()
stats(bin_mat_sparse.toarray())

## Extraction

In [ ]:
netsci_motif_keys = [12, 36, 6, 38, 14, 74, 98, 78, 102, 46, 108, 110, 238]


if os.path.exists(f'{NETWORK}/binary_fsl.json'):
    print('Loading pre-calculated motifs...')
    network_fsl = json.load(open(f'{NETWORK}/binary_fsl.json'))
    network_fsl = {int(k): v for k, v in network_fsl.items()}
else:
    print('Calculating motifs...')
    n_reals, participating_nodes = nsm.motifs(bin_mat_sparse, algorithm='louzoun', participation=True)
    n_reals = n_reals[3:]
    participating_nodes = participating_nodes[3:]

    network_fsl = {netsci_motif_keys[i]: amount for (i, amount) in enumerate(n_reals)}
    fsl_fully_mapped = {netsci_motif_keys[i]: nodes for (i, nodes) in enumerate(participating_nodes)}

    with open(f'{NETWORK}/binary_fsl.json', 'w') as f:
        json.dump({k: int(v) for k, v in network_fsl.items()}, f)

    save_motif_participation_nodes_h5(fsl_fully_mapped, f'{NETWORK}/participation_nodes.h5')